# Creación del modelo

In [1]:
#Carga de librerias
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [2]:
#En los siguientes se carga y limpia la base de datos
datos1 = pd.read_excel('Variabilización.xlsx', sheet_name='Union')

In [3]:
datos1 = datos1.iloc[:, [1,2]]

In [4]:
datos1 = datos1.dropna (axis=0)

In [5]:
datos1=datos1[datos1.duplicated(["target"], keep=False)]
datos1.head()

,text,target
0,Amortiguador golpe de ariete,Amortiguador Golpe de Ariete
1,cale,Calentador de Agua
2,Calentador de Agua CPG 10 TN GN Blanco,Calentador de Agua
3,Zprueba Calentador de agua 6 litros Haceb de p...,Calentador de Agua
4,"Cheque de cortina 3/4"" x 3/4""",Cheque de Cortina


In [6]:
#Se crea la tokenizacion
import string
punctuation = set(string.punctuation)

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [7]:
datos1["text"].apply(tokenize)[:10]

0                    [amortiguador, golpe, de, ariete]
1                                               [cale]
2      [calentador, de, agua, cpg, 10, tn, gn, blanco]
3    [zprueba, calentador, de, agua, 6, litros, hac...
4                     [cheque, de, cortina, 34, x, 34]
5                         [rejilla, piso, rectangular]
6                      [valvula, de, bola, cheque, 12]
7                      [valvula, de, bola, filtro, 12]
8      [papel, higiénico, jumbo, 170m, blanco, ht, x4]
9     [papel, higiénico, jumbo, 400m, natural, hs, x4]
Name: text, dtype: object

In [8]:
#Se carga libreria del modelo
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
#Se parte la base de datos en archivos de prueba y de entrenamiento
from sklearn.model_selection import train_test_split
train_text,test_text, train_labels, test_labels = train_test_split(datos1["text"], 
                                                                    datos1["target"],
                                                                    stratify=datos1["target"])
print(f"Training examples: {len(train_text)}, testing examples {len(test_text)}")

Training examples: 28293, testing examples 9432


In [10]:
#Se vectoriza
vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)

train_X = vectorizer.fit_transform(train_text)
test_X = vectorizer.transform(test_text)

train_X.shape

(28293, 8174)

In [11]:
#Se crea la maquina de soporte vectorial
from sklearn.svm import LinearSVC
classifier = LinearSVC(max_iter=1200000)
classifier.fit(train_X, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1200000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [12]:
#Eficiencia del modelo
from sklearn.metrics import accuracy_score

predicciones = classifier.predict(test_X)

accuracy = accuracy_score(test_labels, predicciones)

print(f"Accuracy: {accuracy:.4%}")

Accuracy: 99.1836%


In [13]:
pd.DataFrame(predicciones)[:10]

,0
0,MALLA EXPANDI IMT 20 1.00X2.00 CR18
1,Malla Expandida
2,Conjunto De Entrada Ppal. Schlage
3,Viga
4,Malla Electrosol
5,Bobina
6,Malla Eslab COMPAC
7,Reduc Tca Galv
8,Codo C-PVC Hot Pro
9,Bordillo de Confinamiento


In [14]:
test_text[:10]

48295                  MALLA EXPANDI IMT 20 1.00X2.00 CR18
48174                 MALLA EXP. IM20 2.50 mm 1000x2000 mm
37233    Conjunto De Entrada Ppal. Schlage Hc387 Centur...
48723                   VIGA HEA220 - 210X220 ESP 7.00 6Mt
38229     Malla Electrosol. xx-221 (6.5 x 6.5mm - 15 x 15)
44978                          BOBINA GAL. 1.85 mm 1220 mm
47871                   MALLA ESLAB C10 2"X2.00X23m COMPAC
38782                       Reduc Tca Galv 1 1/2" X 1 1/4"
34095                            M/Codo Cpvc De 1" Hot Pro
32945    BORDILLO DE CONFINAMIENTO TIPO IDU A81 (35x15x80)
Name: text, dtype: object

In [15]:
examples = ["Kyocera Tk-512K","Etiqueta Adhesiva 102Mmx25Mm C1R2000","Memoria Ram Ddr4 4Gb Cl17 1.2V Kingston"]
examples_X = vectorizer.transform(examples)
prediccion = classifier.predict(examples_X)
prediccion

array(['Kyocera', 'Etiqueta Adhesiva', 'Memoria Ram Ddr4 1.2V Kingston'],
      dtype=object)

In [16]:
#Se cargar archivos de mapeos
datos2 = pd.read_excel('Mapeo2.xlsx', sheet_name='Sheet1')

In [17]:
atributos = pd.DataFrame(columns=('Mapeo','Atributo 1','Atributo 2'))
for clave in prediccion:
    atributos.loc[len(atributos)]=[datos2[datos2["Nombre del Producto"]==clave]["Nombre del Producto"].tolist(),
                                   str(datos2[datos2["Nombre del Producto"]==clave]["Atributo 1"].tolist()),
                                   str(datos2[datos2["Nombre del Producto"]==clave]["Atributo 2"].tolist())] 

In [18]:
tabla = pd.DataFrame({'Ingresado':examples,'Clasificacion':prediccion})
tabla['Atributo 1'] = atributos['Atributo 1']
tabla['Atributo 2'] = atributos['Atributo 2'] 
tabla

,Ingresado,Clasificacion,Atributo 1,Atributo 2
0,Kyocera Tk-512K,Kyocera,['Referencia'],['Sin_atributo']
1,Etiqueta Adhesiva 102Mmx25Mm C1R2000,Etiqueta Adhesiva,['Dimensión'],['Sin_atributo']
2,Memoria Ram Ddr4 4Gb Cl17 1.2V Kingston,Memoria Ram Ddr4 1.2V Kingston,['Memoria'],['Referencia']


## Pruebas con documento

In [19]:
datos2 = pd.read_excel('Mapeo2.xlsx', sheet_name='Sheet1')
prueba = pd.read_excel('Ejemplo_Carga_Productos.xlsx', sheet_name='Sheet1')
proveedor = prueba["Nombre Proveedor"].unique()
proveedor

array(['GRUPO DECOR S.A.S.', 'CERAMICA ITALIA S.A', 'FIRPLAK S.A.',
       'Corona - ColCÃ©ramica SAS', 'ALMACENES CORONA SAS',
       'BOCCHERINI SA', 'INTERDEKO S.A.S', 'ALFAGRES S.A.'], dtype=object)

In [20]:
for prov in proveedor:
    
    #Se vectoriza y se usa predice
    prueba_X = vectorizer.transform(prueba[prueba['Nombre Proveedor']==prov]['Nombre de Producto'].str.strip())
    prediccion_prueba = classifier.predict(prueba_X)
    

    
    #Se mapea
    
    atributos = pd.DataFrame(columns=('Mapeo','Atributo 1','Atributo 2'))
    for clave in prediccion_prueba:
        clave = clave.strip()
        atributos.loc[len(atributos)]=[datos2[datos2["Nombre del Producto"]==clave]["Nombre del Producto"].tolist(),
                                       str(datos2[datos2["Nombre del Producto"]==clave]["Atributo 1"].tolist()),
                                       str(datos2[datos2["Nombre del Producto"]==clave]["Atributo 2"].tolist())]
    #Se crea la tabla final
    
    tabla = pd.DataFrame({'ID de Bimbau':prueba[prueba['Nombre Proveedor']==prov]['ID de Bimbau'],
                          'Texto_ingresado':prueba[prueba['Nombre Proveedor']==prov]['Nombre de Producto'],
                          'Nombre del Producto':prediccion_prueba})           

    tabla['Atributo 1'] = atributos['Atributo 1'].tolist()
    tabla['Atributo 2'] = atributos['Atributo 2'].tolist() 
    
    tabla.to_excel('C:/Users/julia/Desktop/Julius/BIMBAU/Variantes/Creacion de modelos/Resultados transformados/'+prov+'.xlsx', sheet_name=prov)
    
    del tabla

### El siguiente es para crear un archivo unificado

In [21]:
prueba[prueba['Nombre Proveedor']==proveedor[0]]['Nombre de Producto'].str.strip()

0        Griferia lavamanos monoc loft alta
1         Griferia lavamanos a pared duo cr
36                      Orinal steward 4918
52                           Sanitario reve
55               Grif Lavam MC Alta Loft NR
56                       Sanit Pz Elong Zen
57    Griferia lavaplatos monocontrol urban
58                          Sanitario reach
59                 Sanitario san raphael pl
60                      Sanitario san souci
61                           Sanitario numi
62             Sanitario san souci c-sensor
63                   Sanitario eco misty ll
64                      Orinal steward 4917
68              Griferia lavaplatos sensate
69                    Sanitario san raphael
70                     Sanitario santa rosa
Name: Nombre de Producto, dtype: object

In [22]:
prueba_X = vectorizer.transform(prueba['Nombre de Producto'].str.strip())
prediccion_prueba = classifier.predict(prueba_X)

In [23]:
prediccion_prueba[0]

'Griferia Lavamanos monoc loft alta'

In [24]:
str(datos2[datos2["Nombre del Producto"]==prediccion_prueba[0]]["Atributo 1"].tolist()[0])

'Sin_atributo'

In [25]:
datos2 = pd.read_excel('Mapeo2.xlsx', sheet_name='Sheet1')

In [26]:
atributos = pd.DataFrame(columns=('Mapeo','Atributo 1','Atributo 2'))
for clave in prediccion_prueba:
    clave = clave.strip()
    atributos.loc[len(atributos)]=[datos2[datos2["Nombre del Producto"]==clave]["Nombre del Producto"].tolist(),
                                   str(datos2[datos2["Nombre del Producto"]==clave]["Atributo 1"].tolist()[0]),
                                   str(datos2[datos2["Nombre del Producto"]==clave]["Atributo 2"].tolist()[0])]

In [27]:
tabla = pd.DataFrame({'ID de Bimbau':prueba['ID de Bimbau'],'Texto_ingresado':prueba['Nombre de Producto'],
                      'Nombre del Producto':prediccion_prueba,'Nombre del proveedor':prueba['Nombre Proveedor']})
tabla['Atributo 1'] = atributos['Atributo 1']
tabla['Atributo 2'] = atributos['Atributo 2']
tabla

,ID de Bimbau,Texto_ingresado,Nombre del Producto,Nombre del proveedor,Atributo 1,Atributo 2
0,79,Griferia lavamanos monoc loft alta,Griferia Lavamanos monoc loft alta,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo
1,78,Griferia lavamanos a pared duo cr,Griferia Lavamanos a pared duo cr,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo
2,954,Sanitario Fratelo MJ,Sanitario,CERAMICA ITALIA S.A,Tipo,Color
3,952,Sanitario Granada,Sanitario,CERAMICA ITALIA S.A,Tipo,Color
4,1634,Trinsic Grifo de bañoo con dos manijas,Trinsic Grifo de Baño,FIRPLAK S.A.,No. Manijas,Sin_atributo
5,1639,Zura Grifo de baño con dos manijas,Zura Grifo de Baño,FIRPLAK S.A.,No. Manijas,Sin_atributo
6,1640,Mandolin Llave de cocina con juego central de ...,Mandolin Llave de Cocina con Juego Central Una...,FIRPLAK S.A.,Sin_atributo,Sin_atributo
7,1642,Elemetro Llave de cocina de palanca,Elemetro Llave de cocina de palanca,FIRPLAK S.A.,Sin_atributo,Sin_atributo
8,1644,Filis Grifo del lavabo de una sola manija,Filis Grifo del lavabo de una sola manija,FIRPLAK S.A.,Sin_atributo,Sin_atributo
9,1645,Grifo del lavabo de una sola manija,Grifo,FIRPLAK S.A.,No. Manijas,Sin_atributo


## Creación segundo modelo Prueba 1 para un solo atributo

In [28]:
#Carga de librerias
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [29]:
datos3 = pd.read_excel('Archivo_prueba_calibre.xlsx', sheet_name='Sheet1')

In [30]:
datos3 = datos3.iloc[:, [0,3]]

In [31]:
datos3 = datos3.dropna (axis=0)

In [32]:
datos3=datos3[datos3.duplicated(["Valor 1"], keep=False)]
datos3.head()

,Texto,Valor 1
0,A Cu DDO BLANDO 6 AWG,6 AWG
1,A Cu DDO BLANDO 8 AWG,8 AWG
2,A Cu DDO BLANDO 10 AWG,10 AWG
3,A Cu DDO BLANDO 12 AWG,12 AWG
4,A Cu DDO BLANDO 14 AWG,14 AWG


In [33]:
import string
punctuation = set(string.punctuation)

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [34]:
datos3["Texto"].apply(tokenize)[:10]

0             [a, cu, ddo, blando, 6, awg]
1             [a, cu, ddo, blando, 8, awg]
2            [a, cu, ddo, blando, 10, awg]
3            [a, cu, ddo, blando, 12, awg]
4            [a, cu, ddo, blando, 14, awg]
6       [a, cu, ddo, blando, 12, awgx100m]
7       [a, cu, ddo, blando, 14, awgx100m]
8     [a, cu, ddo, blando, 10, awg, x200m]
9     [a, cu, ddo, blando, 12, awg, x200m]
10    [a, cu, ddo, blando, 14, awg, x200m]
Name: Texto, dtype: object

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
#Se parte la base de datos en archivos de prueba y de entrenamiento
from sklearn.model_selection import train_test_split
train_text,test_text, train_labels, test_labels = train_test_split(datos3["Texto"], 
                                                                    datos3["Valor 1"])
print(f"Training examples: {len(train_text)}, testing examples {len(test_text)}")

Training examples: 357, testing examples 119


In [37]:
#Se vectoriza
vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)

train_X = vectorizer.fit_transform(train_text)
test_X = vectorizer.transform(test_text)

train_X.shape

(357, 271)

In [38]:
#Se crea la maquina de soporte vectorial
from sklearn.svm import LinearSVC
classifier = LinearSVC(max_iter=1200000)
classifier.fit(train_X, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1200000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [39]:
#Eficiencia del modelo
from sklearn.metrics import accuracy_score

predicciones = classifier.predict(test_X)

accuracy = accuracy_score(test_labels, predicciones)

print(f"Accuracy: {accuracy:.4%}")

Accuracy: 75.6303%


In [40]:
pd.DataFrame(predicciones)[:10]

,0
0,14 AWG
1,12 AWG
2,14 AWG
3,10 AWG
4,10 AWG
5,6 AWG
6,2/0 AWG
7,8 AWG
8,500 kcmil
9,2 AWG


In [41]:
examples = ["A THHN/THWN-2 CT 17 AWG Cu 600V 90°C NGO","C ACSR 6/0 AWG QUAIL",
            "C MT 120 AAC XLPE 8,7/15 kV (17,5) 90°C CPR PH PVC SR","C TERMOFLEX 4x14 AWG Cu (FLEX) PVC-NY 600V 90°C PVC SR",
           "Dry 100 - Cal .25"]
examples_X = vectorizer.transform(examples)
prediccion = classifier.predict(examples_X)
prediccion

array(['10 AWG', '2/0 AWG', '1/0 AWG', '4AWG', '.25'], dtype=object)

Lo anterior fue realizado para un único atributo separado de forma manual. La idea es optimizar el proceso, de tal forma que una vez se obtenga la primera clasificación se filtre en un segundo archivo, se cree modelo y se proceda a una reclasificación

In [42]:
#Cargamos los datos para el segundo modelo y combinamos en una sola columna los atributos
#para facilitar la busqueda

datos4_temp1 = pd.read_excel('DataModelo2.xlsx', sheet_name='Sheet1')
datos4_temp1 = datos4_temp1.iloc[:, [0,2,3]]
datos4_temp1 = datos4_temp1.rename(columns={'Texto':'Texto','Atributo 1':'Atributo','Valor 1':'Valor'})

datos4_temp2 = pd.read_excel('DataModelo2.xlsx', sheet_name='Sheet1')
datos4_temp2 = datos4_temp2.iloc[:, [0,4,5]]
datos4_temp2 = datos4_temp2.rename(columns={'Texto':'Texto','Atributo 2':'Atributo','Valor 2':'Valor'})

datos4 = pd.concat([datos4_temp1,datos4_temp2])
datos4.head()

,Texto,Atributo,Valor
0,"""Bisagra Omega con balinera Acero Inox 4""",Longitud,4in
1,"""Bisagra Omega In 3"""" Zincada""",Longitud,3in
2,"""N"" Ladrillo Farol Ph 3h De 10x20 Medio Rayado...",Sin_atributo,-
3,"""N"" Ladrillo Farol Ph 3h De 10x40 Rayado Rojo ...",Sin_atributo,-
4,"""N"" Ladrillo Farol Ph 6h De 10x20 Medio Rayado...",Sin_atributo,-


In [43]:
tabla1 = tabla["Nombre del proveedor"].unique()
busquedas1 = tabla[tabla["Nombre del proveedor"]==tabla1[0]]["Atributo 1"].unique()
busquedas2 = tabla[tabla["Nombre del proveedor"]==tabla1[0]]["Atributo 2"].unique()
busquedas=np.concatenate((busquedas1,busquedas2),axis=0)
busquedas

array(['Sin_atributo', 'Referencia', 'Tipo', 'Sin_atributo', 'Color'],
      dtype=object)

In [44]:
proveedores = tabla["Nombre del proveedor"].unique()
busquedas1 = tabla[tabla["Nombre del proveedor"]==proveedores[0]]["Atributo 1"].unique()
busquedas2 = tabla[tabla["Nombre del proveedor"]==proveedores[0]]["Atributo 2"].unique()
busquedas1[1].replace("['","").replace("']","")=='Sin_atributo'

False

In [45]:
proveedores = tabla["Nombre del proveedor"].unique()

busquedas1 = tabla[tabla["Nombre del proveedor"]==proveedores[0]]["Atributo 1"].unique()
busquedas2 = tabla[tabla["Nombre del proveedor"]==proveedores[0]]["Atributo 2"].unique()
#busquedas=np.concatenate((busquedas1,busquedas2),axis=0)
print(proveedores[0]+' '+busquedas1[0])

if (busquedas1[0].replace("['","").replace("']","")!='Sin_atributo'):
    datos5 = datos4[datos4["Atributo"]==busquedas1[0].replace("['","").replace("']","")]
    datos5 = datos5.dropna (axis=0)
    #datos5=datos5[datos5.duplicated(["Valor"], keep=False)]

    import string
    punctuation = set(string.punctuation)

    def tokenize(sentence):
        tokens = []
        for token in sentence.split():
            new_token = []
            for character in token:
                if character not in punctuation:
                    new_token.append(character.lower())
            if new_token:
                tokens.append("".join(new_token))
        return tokens

    datos5["Texto"].apply(tokenize)[:3]

    from sklearn.feature_extraction.text import CountVectorizer

    from sklearn.model_selection import train_test_split
    train_text,test_text, train_labels, test_labels = train_test_split(datos5["Texto"], 
                                                                    datos5["Valor"])
    print(f"Training examples: {len(train_text)}, testing examples {len(test_text)}")

    vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)
    train_X = vectorizer.fit_transform(train_text)
    test_X = vectorizer.transform(test_text)
    print(train_X.shape)
    
 
    from sklearn.svm import LinearSVC
    classifier = LinearSVC(max_iter=1200000)
    classifier.fit(train_X, train_labels)

    from sklearn.metrics import accuracy_score

    predicciones = classifier.predict(test_X)

    accuracy = accuracy_score(test_labels, predicciones)
    aic = accuracy
    print(f"Accuracy: {aic:.4%}")


    #Se vectoriza y se usa predice
    tabla1 = tabla[(tabla['Nombre del proveedor']==proveedores[0]) & (tabla['Atributo 1']==busquedas[0])]
    print(tabla1)
    prueba_X = vectorizer.transform(tabla[(tabla['Nombre del proveedor']==proveedores[0]) & (tabla['Atributo 1']==busquedas1[0]) & (tabla['Atributo 2']==busquedas2[0])]['Texto_ingresado'].str.strip())
    prediccion_prueba = classifier.predict(prueba_X)
    print(prediccion_prueba)
    
    tabla1['Valor 1']= prediccion_prueba
else:
    print('era sin atributo')
    
    tabla1 = tabla[(tabla['Nombre del proveedor']==proveedores[0]) & (tabla['Atributo 1']==busquedas1[0]) & (tabla['Atributo 2']==busquedas2[0])]
    tabla1['Valor 1'] = '-'


#---------------------------------------------------------------------------------------------------

#busquedas = tabla[tabla["Nombre del proveedor"]==proveedores[0]]["Atributo 1"].unique()
busquedas2 = tabla[tabla["Nombre del proveedor"]==proveedores[0]]["Atributo 2"].unique()
#busquedas=np.concatenate((busquedas1,busquedas2),axis=0)
print(proveedores[0]+' '+busquedas2[0])

if (busquedas2[0].replace("['","").replace("']","")!='Sin_atributo'):
    
    datos6 = datos4[datos4["Atributo"]==busquedas2[0].replace("['","").replace("']","")]
    datos6 = datos6.dropna (axis=0)
    #datos5=datos5[datos5.duplicated(["Valor"], keep=False)]

    import string
    punctuation = set(string.punctuation)

    def tokenize(sentence):
        tokens = []
        for token in sentence.split():
            new_token = []
            for character in token:
                if character not in punctuation:
                    new_token.append(character.lower())
            if new_token:
                tokens.append("".join(new_token))
        return tokens

    datos5["Texto"].apply(tokenize)[:3]

    from sklearn.feature_extraction.text import CountVectorizer

    from sklearn.model_selection import train_test_split
    train_text,test_text, train_labels, test_labels = train_test_split(datos6["Texto"], 
                                                                    datos6["Valor"])
    print(f"Training examples: {len(train_text)}, testing examples {len(test_text)}")

    vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)
    train_X = vectorizer.fit_transform(train_text)
    test_X = vectorizer.transform(test_text)
    print(train_X.shape)


    from sklearn.svm import LinearSVC
    classifier = LinearSVC(max_iter=1200000)
    classifier.fit(train_X, train_labels)

    from sklearn.metrics import accuracy_score

    predicciones = classifier.predict(test_X)

    accuracy = accuracy_score(test_labels, predicciones)

    print(f"Accuracy: {accuracy:.4%}")

    #Se vectoriza y se usa predice
    tabla2 = tabla[(tabla['Nombre del proveedor']==proveedores[0]) &  (tabla['Atributo 1']==busquedas1[0]) & (tabla['Atributo 2']==busquedas2[0])]
    print(tabla2)
    prueba_X1 = vectorizer.transform(tabla[(tabla['Nombre del proveedor']==proveedores[0]) &  (tabla['Atributo 1']==busquedas1[0]) & (tabla['Atributo 2']==busquedas2[0])]['Texto_ingresado'].str.strip())
    prediccion_prueba1 = classifier.predict(prueba_X1)
    print(prediccion_prueba1)
    
    tabla2['Valor 2'] = prediccion_prueba1
else:
    print('era un sin atributo')
    
    tabla2 = tabla[(tabla['Nombre del proveedor']==proveedores[0]) &  (tabla['Atributo 1']==busquedas1[0]) & (tabla['Atributo 2']==busquedas2[0])]
    tabla2['Valor 2'] = '-'
    
tabla3 = pd.merge(tabla1,tabla2.iloc[:, [0,6]], on='ID de Bimbau')
print(tabla3)


GRUPO DECOR S.A.S. Sin_atributo
era sin atributo
GRUPO DECOR S.A.S. Sin_atributo
era un sin atributo
   ID de Bimbau                     Texto_ingresado  \
0            79  Griferia lavamanos monoc loft alta   
1            78   Griferia lavamanos a pared duo cr   

                  Nombre del Producto Nombre del proveedor    Atributo 1  \
0  Griferia Lavamanos monoc loft alta   GRUPO DECOR S.A.S.  Sin_atributo   
1   Griferia Lavamanos a pared duo cr   GRUPO DECOR S.A.S.  Sin_atributo   

     Atributo 2 Valor 1 Valor 2  
0  Sin_atributo       -       -  
1  Sin_atributo       -       -  


C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [46]:
tabla2

,ID de Bimbau,Texto_ingresado,Nombre del Producto,Nombre del proveedor,Atributo 1,Atributo 2,Valor 2
0,79,Griferia lavamanos monoc loft alta,Griferia Lavamanos monoc loft alta,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo,-
1,78,Griferia lavamanos a pared duo cr,Griferia Lavamanos a pared duo cr,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo,-


### Haciendo lo anterior  iterando en los provedores y en las busquedas

In [47]:
#Cargamos los datos para el segundo modelo y combinamos en una sola columna los atributos
#para facilitar la busqueda

datos4_temp1 = pd.read_excel('DataModelo2.xlsx', sheet_name='Sheet1')
datos4_temp1 = datos4_temp1.iloc[:, [0,2,3]]
datos4_temp1 = datos4_temp1.rename(columns={'Texto':'Texto','Atributo 1':'Atributo','Valor 1':'Valor'})

datos4_temp2 = pd.read_excel('DataModelo2.xlsx', sheet_name='Sheet1')
datos4_temp2 = datos4_temp2.iloc[:, [0,4,5]]
datos4_temp2 = datos4_temp2.rename(columns={'Texto':'Texto','Atributo 2':'Atributo','Valor 2':'Valor'})

datos4 = pd.concat([datos4_temp1,datos4_temp2])
datos4.head()

,Texto,Atributo,Valor
0,"""Bisagra Omega con balinera Acero Inox 4""",Longitud,4in
1,"""Bisagra Omega In 3"""" Zincada""",Longitud,3in
2,"""N"" Ladrillo Farol Ph 3h De 10x20 Medio Rayado...",Sin_atributo,-
3,"""N"" Ladrillo Farol Ph 3h De 10x40 Rayado Rojo ...",Sin_atributo,-
4,"""N"" Ladrillo Farol Ph 6h De 10x20 Medio Rayado...",Sin_atributo,-


In [48]:
proveedores = tabla["Nombre del proveedor"].unique()

columns2= ['ID de Bimbau','Texto_ingresado','Nombre del Producto','Nombre del proveedor','Atributo 1','Atributo 2','Valor 1','Valor 2']
tabla6 = pd.DataFrame(columns=columns2)

for proveedor in proveedores:
    busquedas1 = tabla[tabla["Nombre del proveedor"]==proveedor]["Atributo 1"].unique()
    busquedas2 = tabla[tabla["Nombre del proveedor"]==proveedor]["Atributo 2"].unique()

    columns= ['ID de Bimbau','Texto_ingresado','Nombre del Producto','Nombre del proveedor','Atributo 1','Atributo 2','Valor 1']
    tabla3 = pd.DataFrame(columns=columns) 

    for busqueda1 in busquedas1:

        print(proveedor+' '+busqueda1)

        if (busqueda1.replace("['","").replace("']","")!='Sin_atributo'):
            datos5 = datos4[datos4["Atributo"]==busqueda1.replace("['","").replace("']","")]
            datos5 = datos5.dropna (axis=0)

            import string
            punctuation = set(string.punctuation)

            def tokenize(sentence):
                tokens = []
                for token in sentence.split():
                    new_token = []
                    for character in token:
                        if character not in punctuation:
                            new_token.append(character.lower())
                    if new_token:
                        tokens.append("".join(new_token))
                return tokens

            datos5["Texto"].apply(tokenize)[:3]

            from sklearn.feature_extraction.text import CountVectorizer

            from sklearn.model_selection import train_test_split
            train_text,test_text, train_labels, test_labels = train_test_split(datos5["Texto"], 
                                                                            datos5["Valor"])
            print(f"Training examples: {len(train_text)}, testing examples {len(test_text)}")

            vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)
            train_X = vectorizer.fit_transform(train_text)
            test_X = vectorizer.transform(test_text)
            print(train_X.shape)


            from sklearn.svm import LinearSVC
            classifier = LinearSVC(max_iter=1200000)
            classifier.fit(train_X, train_labels)

            from sklearn.metrics import accuracy_score

            predicciones = classifier.predict(test_X)

            accuracy = accuracy_score(test_labels, predicciones)
            aic = accuracy
            print(f"Accuracy: {aic:.4%}")


            #Se vectoriza y se usa predice
            tabla1 = tabla[(tabla['Nombre del proveedor']==proveedor) & (tabla['Atributo 1']==busqueda1)]
            print(tabla1)
            prueba_X = vectorizer.transform(tabla[(tabla['Nombre del proveedor']==proveedor) & (tabla['Atributo 1']==busqueda1)]['Texto_ingresado'].str.strip())
            prediccion_prueba = classifier.predict(prueba_X)
            print(prediccion_prueba)

            tabla1['Valor 1']= prediccion_prueba
            tabla3 = tabla3.append(tabla1)
        else:
            print('era sin atributo')

            tabla1 = tabla[(tabla['Nombre del proveedor']==proveedor) & (tabla['Atributo 1']==busqueda1)]
            tabla1['Valor 1'] = '-'
            tabla3 = tabla3.append(tabla1)
    #--------------------------------------------------------------------------------------------------------------------

    columns1= ['ID de Bimbau','Texto_ingresado','Nombre del Producto','Nombre del proveedor','Atributo 1','Atributo 2','Valor 2']
    tabla4 = pd.DataFrame(columns=columns1)

    for busqueda2 in busquedas2:

        print(proveedor+' '+busqueda2)

        if (busqueda2.replace("['","").replace("']","")!='Sin_atributo'):

            datos6 = datos4[datos4["Atributo"]==busqueda2.replace("['","").replace("']","")]
            datos6 = datos6.dropna (axis=0)

            import string
            punctuation = set(string.punctuation)

            def tokenize(sentence):
                tokens = []
                for token in sentence.split():
                    new_token = []
                    for character in token:
                        if character not in punctuation:
                            new_token.append(character.lower())
                    if new_token:
                        tokens.append("".join(new_token))
                return tokens

            datos6["Texto"].apply(tokenize)[:3]

            from sklearn.feature_extraction.text import CountVectorizer

            from sklearn.model_selection import train_test_split
            train_text,test_text, train_labels, test_labels = train_test_split(datos6["Texto"], 
                                                                            datos6["Valor"])
            print(f"Training examples: {len(train_text)}, testing examples {len(test_text)}")

            vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)
            train_X = vectorizer.fit_transform(train_text)
            test_X = vectorizer.transform(test_text)
            print(train_X.shape)


            from sklearn.svm import LinearSVC
            classifier = LinearSVC(max_iter=1200000)
            classifier.fit(train_X, train_labels)

            from sklearn.metrics import accuracy_score

            predicciones = classifier.predict(test_X)

            accuracy = accuracy_score(test_labels, predicciones)

            print(f"Accuracy: {accuracy:.4%}")

            #Se vectoriza y se usa predice
            tabla2 = tabla[(tabla['Nombre del proveedor']==proveedor) & (tabla['Atributo 2']==busqueda2)]
            print(tabla2)
            prueba_X1 = vectorizer.transform(tabla[(tabla['Nombre del proveedor']==proveedor) & (tabla['Atributo 2']==busqueda2)]['Texto_ingresado'].str.strip())
            prediccion_prueba1 = classifier.predict(prueba_X1)
            print(prediccion_prueba1)

            tabla2['Valor 2'] = prediccion_prueba1
            tabla4 = tabla4.append(tabla2)
        else:
            print('era un sin atributo')

            tabla2 = tabla[(tabla['Nombre del proveedor']==proveedor)  & (tabla['Atributo 2']==busqueda2)]
            tabla2['Valor 2'] = '-'
            tabla4 = tabla4.append(tabla2)

    tabla5 = pd.merge(tabla3,tabla4.iloc[:, [0,6]], on='ID de Bimbau')
    tabla6 = tabla6.append(tabla5)    

GRUPO DECOR S.A.S. Sin_atributo
era sin atributo
GRUPO DECOR S.A.S. Referencia
Training examples: 425, testing examples 142
(425, 965)


C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Accuracy: 35.9155%
    ID de Bimbau      Texto_ingresado Nombre del Producto  \
36            74  Orinal steward 4918      Orinal Steward   
64            51  Orinal steward 4917      Orinal Steward   

   Nombre del proveedor  Atributo 1    Atributo 2  
36   GRUPO DECOR S.A.S.  Referencia  Sin_atributo  
64   GRUPO DECOR S.A.S.  Referencia  Sin_atributo  
['4918.' '4917.']
GRUPO DECOR S.A.S. Tipo
Training examples: 1783, testing examples 595


C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(1783, 2037)
Accuracy: 51.9328%
    ID de Bimbau                        Texto_ingresado  \
52            90                         Sanitario reve   
55         21461             Grif Lavam MC Alta Loft NR   
56         21463                     Sanit Pz Elong Zen   
57            77  Griferia lavaplatos monocontrol urban   
58            89                        Sanitario reach   
59            91               Sanitario san raphael pl   
60            94                    Sanitario san souci   
61            88                         Sanitario numi   
62            93           Sanitario san souci c-sensor   
63          4160                 Sanitario eco misty ll   
68           100            Griferia lavaplatos sensate   
69            92                  Sanitario san raphael   
70            95                   Sanitario santa rosa   

                Nombre del Producto Nombre del proveedor Atributo 1  \
52                        Sanitario   GRUPO DECOR S.A.S.       Tipo   

...


Training examples: 1023, testing examples 342
(1023, 1357)
Accuracy: 70.1754%
    ID de Bimbau               Texto_ingresado Nombre del Producto  \
52            90                Sanitario reve           Sanitario   
55         21461    Grif Lavam MC Alta Loft NR     Grifo Lavamanos   
56         21463            Sanit Pz Elong Zen           Sanitario   
58            89               Sanitario reach           Sanitario   
59            91      Sanitario san raphael pl           Sanitario   
60            94           Sanitario san souci           Sanitario   
61            88                Sanitario numi           Sanitario   
62            93  Sanitario san souci c-sensor           Sanitario   
63          4160        Sanitario eco misty ll           Sanitario   
69            92         Sanitario san raphael           Sanitario   
70            95          Sanitario santa rosa           Sanitario   

   Nombre del proveedor Atributo 1 Atributo 2  
52   GRUPO DECOR S.A.S.       Ti

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training examples: 1783, testing examples 595
(1783, 2035)
Accuracy: 53.6134%
    ID de Bimbau              Texto_ingresado Nombre del Producto  \
2            954         Sanitario Fratelo MJ           Sanitario   
3            952            Sanitario Granada           Sanitario   
71         52182  Sanitario Danubio MJ Blanco           Sanitario   
73         52181     Sanitario Bari MJ Blanco           Sanitario   

   Nombre del proveedor Atributo 1 Atributo 2  
2   CERAMICA ITALIA S.A       Tipo      Color  
3   CERAMICA ITALIA S.A       Tipo      Color  
71  CERAMICA ITALIA S.A       Tipo      Color  
73  CERAMICA ITALIA S.A       Tipo      Color  
['Ecoclean' 'Ecoclean' 'Fussion Elongado' 'Bari']
CERAMICA ITALIA S.A Estilo
Training examples: 42, testing examples 14
(42, 93)
Accuracy: 21.4286%
    ID de Bimbau                   Texto_ingresado Nombre del Producto  \
72         52183  DIAMANTE ROYAL BLANCO RECT 60X60            Cerámica   

   Nombre del proveedor Atributo 1 Atri

FIRPLAK S.A. Color
Training examples: 1023, testing examples 342
(1023, 1365)
Accuracy: 64.6199%
    ID de Bimbau                                    Texto_ingresado  \
26          1670  PRECEPT Grifo Monomando de cocina abatible - N...   
27          1673           XANDER\nValve Only Trim Kit - Negro mate   
28          1674  XANDER\nValve Only MultiChoice - Níquel cepillado   
97          1669  PRECEPT Grifo Monomando de cocina abatible - C...   
98          1671  PRECEPT Grifo Monomando de cocina abatible - A...   
99          1672           XANDER\nValve Only MultiChoice - Cromado   

                           Nombre del Producto Nombre del proveedor  \
26  PRECEPT Grifo Monomando de Cocina Abatible         FIRPLAK S.A.   
27                   XANDER Valve Only TrimKit         FIRPLAK S.A.   
28               XANDER Valve Only MultiChoice         FIRPLAK S.A.   
97  PRECEPT Grifo Monomando de Cocina Abatible         FIRPLAK S.A.   
98  PRECEPT Grifo Monomando de Cocina Abatible    

Training examples: 1023, testing examples 342
(1023, 1353)
Accuracy: 67.2515%
     ID de Bimbau                         Texto_ingresado  \
49          51316    Sanitario Montecarlo Novo Het Rd Bne   
54            489  Sanitario Montecarlo Novo Alongado Het   
116         50633                  SANITARIO SOLARE EN CJ   
135         51322          Sanitario Smart Rd Single Blco   
139         51077                SANITARIO AQUAPRO AL BCO   
141         51083   SANITARIO MONTECARLO NOVO HET AL BLCO   

          Nombre del Producto  Nombre del proveedor Atributo 1 Atributo 2  
49                  Sanitario  ALMACENES CORONA SAS       Tipo      Color  
54                  Sanitario  ALMACENES CORONA SAS       Tipo      Color  
116                 Sanitario  ALMACENES CORONA SAS       Tipo      Color  
135                 Sanitario  ALMACENES CORONA SAS       Tipo      Color  
139  SANITARIO AQUAPRO AL BCO  ALMACENES CORONA SAS       Tipo      Color  
141                 Sanitario  ALMACEN

In [49]:
tabla6.head()

,ID de Bimbau,Texto_ingresado,Nombre del Producto,Nombre del proveedor,Atributo 1,Atributo 2,Valor 1,Valor 2
0,79,Griferia lavamanos monoc loft alta,Griferia Lavamanos monoc loft alta,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo,-,-
1,78,Griferia lavamanos a pared duo cr,Griferia Lavamanos a pared duo cr,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo,-,-
2,74,Orinal steward 4918,Orinal Steward,GRUPO DECOR S.A.S.,Referencia,Sin_atributo,4918.,-
3,51,Orinal steward 4917,Orinal Steward,GRUPO DECOR S.A.S.,Referencia,Sin_atributo,4917.,-
4,90,Sanitario reve,Sanitario,GRUPO DECOR S.A.S.,Tipo,Color,Reve,Negro


In [50]:
filtro=[]
filtro1=[]
filtro2=[]
for i in range(0,len(tabla6)):
    texto = tabla6.iloc[i,1].lower()
    clas = tabla6.iloc[i,6].lower()
    clas2= tabla6.iloc[i,7].lower()
    clas3 = tabla6.iloc[i,2].lower()
    
    atributo1 = tabla6.iloc[i,4].lower()
    atributo2 = tabla6.iloc[i,5].lower()
    
    if texto.find(clas3)==-1:
            filtro.append(0)
    else:
        filtro.append(1)
    
    if tabla6.iloc[i,4].replace("['","").replace("']","")!='Sin_atributo':
        if texto.find(clas)==-1:
            filtro1.append(0)
        else:
            filtro1.append(1)
    else:
        filtro1.append(1)
    
    if tabla6.iloc[i,5].replace("['","").replace("']","")!='Sin_atributo':
        if texto.find(clas2)==-1:
            filtro2.append(0)
        else:
            filtro2.append(1)
    else:
        filtro2.append(1)
tabla6['Filtro'] = filtro
tabla6['Filtro 1'] = filtro1
tabla6['Filtro 2'] = filtro2

In [51]:
tabla6

,ID de Bimbau,Texto_ingresado,Nombre del Producto,Nombre del proveedor,Atributo 1,Atributo 2,Valor 1,Valor 2,Filtro,Filtro 1,Filtro 2
0,79,Griferia lavamanos monoc loft alta,Griferia Lavamanos monoc loft alta,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo,-,-,1,1,1
1,78,Griferia lavamanos a pared duo cr,Griferia Lavamanos a pared duo cr,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo,-,-,1,1,1
2,74,Orinal steward 4918,Orinal Steward,GRUPO DECOR S.A.S.,Referencia,Sin_atributo,4918.,-,1,0,1
3,51,Orinal steward 4917,Orinal Steward,GRUPO DECOR S.A.S.,Referencia,Sin_atributo,4917.,-,1,0,1
4,90,Sanitario reve,Sanitario,GRUPO DECOR S.A.S.,Tipo,Color,Reve,Negro,1,1,0
5,21461,Grif Lavam MC Alta Loft NR,Grifo Lavamanos,GRUPO DECOR S.A.S.,Tipo,Color,Ordinario,Negro,0,0,0
6,21463,Sanit Pz Elong Zen,Sanitario,GRUPO DECOR S.A.S.,Tipo,Color,PZ Elongado,Negro,0,0,0
7,77,Griferia lavaplatos monocontrol urban,Griferia Lavaplatos Monocontrol,GRUPO DECOR S.A.S.,Tipo,Sin_atributo,Urban,-,1,1,1
8,89,Sanitario reach,Sanitario,GRUPO DECOR S.A.S.,Tipo,Color,Reach,Negro,1,1,0
9,91,Sanitario san raphael pl,Sanitario,GRUPO DECOR S.A.S.,Tipo,Color,San Raphael Pl,San Carlos Azul,1,1,0


Ahora es necesario mirar si alguno de los productos puede ser variantizado como bajo consumo

In [52]:
filtro_bajo_consumo = pd.read_excel('filtro_bajo_consumo.xlsx', sheet_name='Sheet1')
nombre_productos = tabla6["Nombre del Producto"]

bajo_consumo = pd.DataFrame(columns=['Bajo consumo'])
for nombre_producto in nombre_productos:
    nombre_producto = nombre_producto.strip()
    if(len(str(filtro_bajo_consumo[filtro_bajo_consumo["productos_bajo_consumo"]==nombre_producto]["atributo"].tolist()).strip())==2):
        bajo_consumo.loc[len(bajo_consumo)]=0
    else:
        bajo_consumo.loc[len(bajo_consumo)]=[str(filtro_bajo_consumo[filtro_bajo_consumo["productos_bajo_consumo"]==nombre_producto]["atributo"].tolist()[0]).strip()]
                                
tabla6['Bajo Consumo']=bajo_consumo['Bajo consumo']
tabla6.head()

,ID de Bimbau,Texto_ingresado,Nombre del Producto,Nombre del proveedor,Atributo 1,Atributo 2,Valor 1,Valor 2,Filtro,Filtro 1,Filtro 2,Bajo Consumo
0,79,Griferia lavamanos monoc loft alta,Griferia Lavamanos monoc loft alta,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo,-,-,1,1,1,L/min
1,78,Griferia lavamanos a pared duo cr,Griferia Lavamanos a pared duo cr,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo,-,-,1,1,1,Lpd
2,74,Orinal steward 4918,Orinal Steward,GRUPO DECOR S.A.S.,Referencia,Sin_atributo,4918.,-,1,0,1,0
3,51,Orinal steward 4917,Orinal Steward,GRUPO DECOR S.A.S.,Referencia,Sin_atributo,4917.,-,1,0,1,0
4,90,Sanitario reve,Sanitario,GRUPO DECOR S.A.S.,Tipo,Color,Reve,Negro,1,1,0,Lpd


In [53]:
atributos_si =[]
id_bimbau_si =[]

descripcion=[]
valor_bajo_consumo=[]

for i in range(0,len(tabla6)):
    atributo = tabla6.iloc[i,11]
    id_bimbau = tabla6.iloc[i,0]
     
    if atributo!=0:
        
        atributos_si.append(atributo)
        id_bimbau_si.append(id_bimbau)
        
        datos1 = pd.read_excel('Entrenamiento bajo consumo.xlsx', sheet_name='Sheet2')
        datos1 = datos1.iloc[:, [5,7]]
        datos1 = datos1.dropna (axis=0)
        
        import string
        punctuation = set(string.punctuation)

        def tokenize(sentence):
            tokens = []
            for token in sentence.split():
                new_token = []
                for character in token:
                    if character not in punctuation:
                        new_token.append(character.lower())
                if new_token:
                    tokens.append("".join(new_token))
            return tokens
        
        #Se parte la base de datos en archivos de prueba y de entrenamiento
        
        #SE HACE RELECTURA DEL ULTIMO DOCUMENTO GENERADO
        from stationarymodel import choose
        escogido_estacionario = rep(a,b,c)
        for a in range(1:100):
            for b in range(1:100):
                for c in range (1:100):
                    from sklearn.model_selection import train_test_split
                    train_text,test_text, train_labels, test_labels = train_test_split(datos1["description"], 
                                                                                    datos1["Valor"],par=[a,b,c])
                    escogido_estaionario= choose(a,b,c)
        
        # Se asegura serie estacionaria
        
        vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)

        train_X = vectorizer.fit_transform(train_text)
        test_X = vectorizer.transform(test_text)
        
        #Se crea la maquina de soporte vectorial CON CICLO
        a = 120000
        if accuracy_score(test_labels, predicciones) <0.70:
            from sklearn.svm import LinearSVC
            classifier = LinearSVC(max_iter=1200000)
            classifier.fit(train_X, train_labels)
            a=a+100
        
        descripcion.append(str(prueba[prueba['ID de Bimbau']==id_bimbau]['Descripción del producto'].tolist()[0]).strip())
        
        examples_X = vectorizer.transform(prueba[prueba['ID de Bimbau']==id_bimbau]['Descripción del producto'].tolist())
        prediccion = classifier.predict(examples_X)
        
        valor_bajo_consumo.append(prediccion.tolist()[0])

mapeo_bajo_consumo = pd.DataFrame({'ID de Bimbau':id_bimbau_si,'Atributo':atributos_si,
                                  'Valor bajo consumo':valor_bajo_consumo, 'Descripcion':descripcion})

In [54]:
tabla6_1 = pd.merge(tabla6,mapeo_bajo_consumo.iloc[:, [0,2]], on='ID de Bimbau',how='left')

In [55]:
tabla6_1
tabla6_1.to_excel('C:/Users/julia/Desktop/Julius/BIMBAU/Variantes/Creacion de modelos/Resultados transformados/Tabla_completa.xlsx', sheet_name='Sheet1')

In [56]:
tabla6_1

,ID de Bimbau,Texto_ingresado,Nombre del Producto,Nombre del proveedor,Atributo 1,Atributo 2,Valor 1,Valor 2,Filtro,Filtro 1,Filtro 2,Bajo Consumo,Valor bajo consumo
0,79,Griferia lavamanos monoc loft alta,Griferia Lavamanos monoc loft alta,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo,-,-,1,1,1,L/min,6.1
1,78,Griferia lavamanos a pared duo cr,Griferia Lavamanos a pared duo cr,GRUPO DECOR S.A.S.,Sin_atributo,Sin_atributo,-,-,1,1,1,Lpd,4.5
2,74,Orinal steward 4918,Orinal Steward,GRUPO DECOR S.A.S.,Referencia,Sin_atributo,4918.,-,1,0,1,0,NaN
3,51,Orinal steward 4917,Orinal Steward,GRUPO DECOR S.A.S.,Referencia,Sin_atributo,4917.,-,1,0,1,0,NaN
4,90,Sanitario reve,Sanitario,GRUPO DECOR S.A.S.,Tipo,Color,Reve,Negro,1,1,0,Lpd,6
5,21461,Grif Lavam MC Alta Loft NR,Grifo Lavamanos,GRUPO DECOR S.A.S.,Tipo,Color,Ordinario,Negro,0,0,0,L/min,6.1
6,21463,Sanit Pz Elong Zen,Sanitario,GRUPO DECOR S.A.S.,Tipo,Color,PZ Elongado,Negro,0,0,0,Lpd,6
7,77,Griferia lavaplatos monocontrol urban,Griferia Lavaplatos Monocontrol,GRUPO DECOR S.A.S.,Tipo,Sin_atributo,Urban,-,1,1,1,L/min,2.8
8,89,Sanitario reach,Sanitario,GRUPO DECOR S.A.S.,Tipo,Color,Reach,Negro,1,1,0,Lpd,6
9,91,Sanitario san raphael pl,Sanitario,GRUPO DECOR S.A.S.,Tipo,Color,San Raphael Pl,San Carlos Azul,1,1,0,Lpd,1


Finalmente se guarda cada uno de los producos en diferentes archivos con el nombre de los proveedores

In [57]:
proveedores = tabla["Nombre del proveedor"].unique()
for prov in proveedores:
    
    tabla7 = tabla6_1[tabla6_1['Nombre del proveedor']==prov]
    
    tabla7.to_excel('C:/Users/julia/Desktop/Julius/BIMBAU/Variantes/Creacion de modelos/Resultados transformados/Bajo consumo/'+prov+'.xlsx', sheet_name='Sheet1')